## Memory
In some applications like chatbots, it is important to remember previous interactions to keep the whole context of a conversation.
Memory does provide you an easy way to handle this

In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
from langchain import PromptTemplate, OpenAI, LLMChain

load_dotenv(find_dotenv())
openai.api_key = os.environ["OPENAI_API_KEY"]

In [2]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()

history.add_user_message("hi!")
history.add_ai_message("hello my friend!")
history.messages

[HumanMessage(content='hi!', additional_kwargs={}, example=False),
 AIMessage(content='hello my friend!', additional_kwargs={}, example=False)]

In [3]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("hello my friend!")
memory.load_memory_variables({})

{'history': 'Human: hi!\nAI: hello my friend!'}

In [4]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain

llm = OpenAI(temperature=0)
conversation = ConversationChain(
    llm=llm, verbose=True, memory=ConversationBufferMemory()
)
conversation.predict(input="Hi")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi
AI:

> Finished chain.


' Hi there! How can I help you?'

In [5]:
conversation.predict(input="I need to know the capital of france")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi
AI:  Hi there! How can I help you?
Human: I need to know the capital of france
AI:

> Finished chain.


' The capital of France is Paris. It is located in the northern part of the country and is the most populous city in France. It is also the cultural and economic center of the country.'

## ConversationSummaryMemory
When inputs get long, we might not want to send the whole conversation, but rather a summary.

In [6]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.9/781.9 kB 20.7 MB/s eta 0:00:0000:01

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [7]:
from langchain.memory import ConversationSummaryBufferMemory

review = "I ordered Pizza Salami for 9.99$ and it was awesome! \
The pizza was delivered on time and was still hot when I received it. \
The crust was thin and crispy, and the toppings were fresh and flavorful. \
The Salami was well-cooked and complemented the cheese perfectly. \
The price was reasonable and I believe I got my money's worth. \
Overall, I am very satisfied with my order and I would recommend this pizza place to others."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context(
    {"input": "Hello, how can I help you today?"},
    {"output": "Could you analyze a review for me?"},
)
memory.save_context(
    {"input": "Sure, I'd be happy to. Could you provide the review?"},
    {"output": f"{review}"},
)

In [8]:
conversation = ConversationChain(llm=llm, memory=memory, verbose=True)

In [9]:
conversation.predict(input="Thank you very much!")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: 
The human asked how they could help the AI, and the AI requested an analysis of a review. The human agreed to provide the review.
AI: I ordered Pizza Salami for 9.99$ and it was awesome! The pizza was delivered on time and was still hot when I received it. The crust was thin and crispy, and the toppings were fresh and flavorful. The Salami was well-cooked and complemented the cheese perfectly. The price was reasonable and I believe I got my money's worth. Overall, I am very satisfied with my order and I would recommend this pizza place to others.
Human: Thank you very much!
AI:

> Finished chain.


" You're welcome! It's always a pleasure to share my experiences. Is there anything else I can help you with?"

In [10]:
memory.load_memory_variables({})

{'history': "System: \n\nThe human asked how they could help the AI, and the AI requested an analysis of a review. The human agreed to provide the review, which was a positive review of a pizza order with Salami for 9.99$. The pizza was delivered on time and was still hot, with a thin and crispy crust and fresh and flavorful toppings. The Salami was well-cooked and complemented the cheese perfectly. The price was reasonable and the customer was very satisfied with their order, recommending the pizza place to others.\nHuman: Thank you very much!\nAI:  You're welcome! It's always a pleasure to share my experiences. Is there anything else I can help you with?"}